In [ ]:
import sys
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
from math import exp
%matplotlib inline

M_PI = 3.1415926535898

'''
def gaussian5x5(cv_img):
    nHeight, nWidth, nChannel = cv_img.shape
    nWStepG = nWStep = nWidth * nChannel
    
    pGss = (np.zeros(shape=(nWidth,nHeight))).flatten()
    
    nWStep2 = 2 * nWStep
    
    print(nWStep)
    
    for r in range (2, nHeight - 2):
        for c in range (2, nWidth - 2):
            print(c)
            print(nWStep2)
            print(nWStep)
            pGss[c] = (2 * (cv_img[c - nWStep2 - 2] + cv_img[c - nWStep2 + 2]
                            + cv_img[c + nWStep2 - 2] + cv_img[c + nWStep2 + 2]) + 4 * 
                       (cv_img[c - nWStep2 - 1] + cv_img[c - nWStep2 + 1] + cv_img[c - nWStep - 2] + cv_img[c - nWStep2]
                        + cv_img[c + nWStep2 - 2] + cv_img[c + nWStep + 2] + cv_img[c + nWStep2 - 1]
                        + cv_img[c + nWStep2 + 1]) + 5 * (cv_img[c - nWStep2] + cv_img[c - 2] + cv_img[c + 2]
                        + cv_img[c + nWStep2]) + 9 * (cv_img[c - nWStep - 1] + cv_img[c - nWStep + 1]
                        + cv_img[c + nWStep - 1] + cv_img[c + nWStep + 1]) + 12 * (cv_img[c - nWStep]
                        + cv_img[c + nWStep] + cv_img[c - 1] + cv_img[c + 1]) + 15 * (cv_img[c])) / 159.0
    
    return pGss
'''

def canny_edge_detecting(cv_img, threshold_LOW, threshold_HIGH):
       
    # Gaussian masking
    pGss = gaussian_2d_filtering(cv_img)
    pGss = cv2.cvtColor(pGss, cv2.COLOR_RGB2GRAY)
    
    nHeight, nWidth = pGss.shape
    nWStepG = nWidth * 1
    
    pGss = pGss.flatten()
    
    # Sobel masking
    imageMag = np.zeros(shape=(nHeight, nWidth))
    imageAng = np.zeros(shape=(nHeight, nWidth))
    
    #pMag = (np.zeros(shape=(nWidth,nHeight))).flatten()
    #pAng = (np.zeros(shape=(nWidth,nHeight))).flatten()
    
    Gx = [-1, 0, 1, -2, 0, 2, -1, 0, 1]
    Gy = [1, 2, 1, 0, 0, 0, -1, -2, -1]
    
    for r in range(0, nHeight - 1):
        pMag = imageMag[r]
        pAng = imageAng[r]
        for c in range(0, nWidth - 1):
            sumX = 0.0;
            sumY = 0.0;
            for y in range(0, 3):
                for x in range(0, 3):
                    py = r - 1 + y
                    px = c - 1 + x
                    
                    if 0 <= px < nWidth and 0 <= py < nHeight:
                        sumX += Gx[y * 3 + x] * pGss[py * nWStepG + px]
                        sumY += Gy[y * 3 + x] * pGss[py * nWStepG + px]
                    
            pMag[c] = math.sqrt(sumX * sumX + sumY * sumY)
            theta = 0.0
            if pMag[c] == 0:
                if sumY == 0:
                    theta = 0
                else:
                    theta = 90
            else:
                theta = math.atan2(sumY, sumX) * 180.0 / M_PI
            
            if (-22.5 < theta < 22.5) or theta > 157.5 or theta < -157.5:
                pAng[c] = 0
            elif (22.5 <= theta < 67.5) or (-157.5 <= theta < -112.5):
                pAng[c] = 45
            elif (67.5 <= theta < 112.5) or (-112.5 <= theta <= -67.5):
                pAng[c] = 90
            else:
                pAng[c] = 135
    
    imageCandidate = np.zeros(shape=(nHeight, nWidth))
    print(nWStepG)
    
    for r in range(1, nHeight - 1):
        pCand = imageCandidate[r]
        pMag = imageMag[r]
        pAng = imageAng[r]
        
        for c in range(1, nWidth - 1):
            # Check degree (0, 45, 90, 135)
            if pAng[c] == 0:
                if pMag[c] > pMag[c - 1] and pMag[c] > pMag[c + 1]:
                    pCand[c] = 255
            elif pAng[c] == 45:
                if pMag[c] > pMag[c - nWStepG + 1] and pMag[c] > pMag[c + nWStepG - 1]:
                    pCand[c] = 255
            elif pAng[c] == 90:
                if pMag[c] > pMag[c - nWStepG] and pMag[c] > pMag[c + nWStepG]:
                    pCand[c] = 255
            elif pAng[c] == 135:
                if pMag[c] > pMag[c - nWStepG - 1] and pMag[c] > pMag[c + nWStepG + 1]:
                    pCand[c] = 255
                    
    # Checking threshold and return result
    imageResult = np.zeros(shape=(nHeight, nWidth))
    for r in range (1, nHeight -1):
        pOut = imageResult[r]
        pMag = imageMag[r]
        pAng = imageAng[r]
        for c in range(1, nWidth - 1):
            if pCand[c] > 0:
                if pMag[c] > threshold_HIGH:
                    pOut[c] = 255
                elif pMag[c] > threshold_LOW:
                    if pAng[c] == 0:
                        if (pMag[c - nWStepG] > threshold_HIGH) or (pMag[c + nWStepG]) > threshold_HIGH:
                            pOut = 255
                    elif pAng[c] == 45:
                        if (pMag[c - nWStepG - 1] > threshold_HIGH) or (pMag[c + nWStepG + 1]) > threshold_HIGH:
                            pOut = 255
                    elif pAng[c] == 90:
                        if (pMag[c - 1] > threshold_HIGH) or (pMag[c + 1] > threshold_HIGH):
                            pOut = 255
                    elif pAng[c] == 135:
                        if (pMag[c-nWStepG + 1] > threshold_HIGH) or (pMag[c + nWStepG - 1] > threshold_HIGH):
                            pOut = 255
    
    #pResult = pResult.reshape(nHeight,nWidth)
    #pResult = pAng.reshape(nHeight,nWidth)
    imageResult = imageResult.astype('uint8')
    
    return imageResult

def gaussian_2d_filtering(cv_img, deviation = 0.4):
    nHeight, nWidth, nChannel = cv_img.shape
    
    m_bufGss = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    
    m_dGaussian = deviation
    
    nHalf = int(max((m_dGaussian * 6 - 1) / 2, 1))
    nWstep = nWidth * nChannel
    
    pIn = cv_img.flatten()
    pTmp = (np.zeros(shape=(nWidth,nHeight,nChannel))).flatten()
    pOut = (np.zeros(shape=(nWidth,nHeight,nChannel))).flatten()
    
    n = 0
    while n <= nHalf:
      m_bufGss[nHalf - n] = m_bufGss[nHalf + n] = exp((-1 * n) * n / (2 * m_dGaussian*m_dGaussian)) 
      n += 1 

    for r in range(0, nHeight):
      for c in range(0, nWidth):
        for l in range(0, nChannel):
          dSum = dGss = 0
    
          for n in range(-1 * nHalf, nHalf+1):
            px = c + n
            
            if 0 <= px < nWidth:
                dSum += (pIn[nWstep * r + nChannel * px + l] * m_bufGss[nHalf + n])
                dGss += m_bufGss[nHalf + n]
          
          pTmp[nWstep * r + nChannel * c + l] = int(dSum / dGss)
    
    for r in range(0, nHeight):
      for c in range(0, nWidth):
        for l in range(0, nChannel):
          dSum = dGss = 0
    
          for n in range(-1 * nHalf, nHalf+1):
            py = r + n
            if 0 <= py < nHeight:
              absN = abs(n)
              dSum += (pTmp[nWstep * py + nChannel * c + l] * m_bufGss[nHalf + absN])
              dGss += m_bufGss[nHalf + absN]
            
          pOut[nWstep * r + nChannel * c + l] = int(dSum / dGss)
    np.set_printoptions(threshold=sys.maxsize)
    
    pResult = pOut.reshape(nHeight,nWidth,nChannel)
    pResult = pResult.astype('uint8')
    
    return pResult

cv_img = cv2.imread('road.jpg')
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

# Implemented gaussian blur ( very slow ) 
pResult = gaussian_2d_filtering(cv_img, 2)

# OpenCV gaussian blur library ( fast & good )
# pResult = cv2.GaussianBlur(cv_img, (9, 9),10)

#pResult = canny_edge_detecting(cv_img, 0, 50)

#plt.imshow(pResult, cmap='gray')

pResult = cv2.cvtColor(pResult, cv2.COLOR_RGB2GRAY)
pResult = cv2.Canny(pResult, 50, 100)

H, W = pResult.shape

mask = np.zeros((H, W))
for i in range(H):
    for j in range(W):
        if i > (H / W) * j and i > -(H / W) * j + H:
            pResult[i, j] = pResult[i, j] * 1 #[i, j] = 1
        else:
            pResult[i, j] = 0

roi = pResult * mask

lines = cv2.HoughLinesP(pResult, 1, np.pi/180, 40)

for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(cv_img, (x1, y1), (x2, y2), (255, 0, 0), 2)

plt.imshow(cv_img, cmap='gray')
